# 0.0 Import Libraries and Loading Dataset Raw

In [1]:
import pandas as pd
import numpy as np
import io
import datetime

In [2]:
df = pd.read_csv('csv/train.csv')

In [19]:
# adjusting tools for better visual
pd.options.display.max_columns = None

# 1.0 Cleaning the Data

In [3]:
print(f"Number of rows    : {df.shape[0]}")
print(f"Number of columns : {df.shape[1]}")

Number of rows    : 45593
Number of columns : 20


In [4]:
df.dtypes

ID                              object
Delivery_person_ID              object
Delivery_person_Age             object
Delivery_person_Ratings         object
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Order_Date                      object
Time_Orderd                     object
Time_Order_picked               object
Weatherconditions               object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries             object
Festival                        object
City                            object
Time_taken(min)                 object
dtype: object

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45593 entries, 0 to 45592
Data columns (total 20 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           45593 non-null  object 
 1   Delivery_person_ID           45593 non-null  object 
 2   Delivery_person_Age          45593 non-null  object 
 3   Delivery_person_Ratings      45593 non-null  object 
 4   Restaurant_latitude          45593 non-null  float64
 5   Restaurant_longitude         45593 non-null  float64
 6   Delivery_location_latitude   45593 non-null  float64
 7   Delivery_location_longitude  45593 non-null  float64
 8   Order_Date                   45593 non-null  object 
 9   Time_Orderd                  45593 non-null  object 
 10  Time_Order_picked            45593 non-null  object 
 11  Weatherconditions            45593 non-null  object 
 12  Road_traffic_density         45593 non-null  object 
 13  Vehicle_conditio

In [7]:
df.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weatherconditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken(min)'],
      dtype='object')

In [8]:
df['Delivery_person_Age'].unique()

array(['37', '34', '23', '38', '32', '22', '33', '35', '36', '21', '24',
       '29', '25', '31', '27', '26', '20', 'NaN ', '28', '39', '30', '15',
       '50'], dtype=object)

In [9]:
df['City'].unique()

array(['Urban ', 'Metropolitian ', 'Semi-Urban ', 'NaN '], dtype=object)

In [12]:
df['Order_Date'].unique()

array(['19-03-2022', '25-03-2022', '05-04-2022', '26-03-2022',
       '11-03-2022', '04-03-2022', '14-03-2022', '20-03-2022',
       '12-02-2022', '13-02-2022', '14-02-2022', '02-04-2022',
       '01-03-2022', '16-03-2022', '15-02-2022', '10-03-2022',
       '27-03-2022', '12-03-2022', '01-04-2022', '05-03-2022',
       '11-02-2022', '08-03-2022', '03-04-2022', '30-03-2022',
       '28-03-2022', '18-03-2022', '06-04-2022', '04-04-2022',
       '24-03-2022', '09-03-2022', '02-03-2022', '13-03-2022',
       '29-03-2022', '31-03-2022', '17-03-2022', '07-03-2022',
       '15-03-2022', '16-02-2022', '03-03-2022', '18-02-2022',
       '23-03-2022', '17-02-2022', '06-03-2022', '21-03-2022'],
      dtype=object)

## 1.1 Making a Copy and Start cleaning the Data

In [52]:
# Copy df to start changes
df1 = df.copy()

In [53]:
# Checked this data has a lot o extra spaces in all columns, triming this spaces
df1 = df1.applymap(lambda x : x.strip() if isinstance(x, str) else x)

# Assuming incosistences in the dataset, we`re dropping all 'NaN' lines
df1 = df1[df1['Delivery_person_Age'] != 'NaN']
df1 = df1[df1['Time_Orderd'] != 'NaN']
df1 = df1[df1['City'] != 'NaN']
df1 = df1[df1['Festival'] != 'NaN']
df1 = df1[df1['multiple_deliveries'] != 'NaN']

# Removing extra info from cells
df1['Weatherconditions'] = df1['Weatherconditions'].str.replace('conditions ', "")
df1['Time_taken(min)'] = df1['Time_taken(min)'].str.slice(start=6)

# Changing dtypes
df1['Delivery_person_Age'] = df1['Delivery_person_Age'].astype('int64')
df1['Delivery_person_Ratings'] = df1['Delivery_person_Ratings'].astype('float64')
df1['Order_Date'] = pd.to_datetime(df1['Order_Date'], dayfirst=True)
df1['Weatherconditions'] = df1['Weatherconditions'].astype('category')
df1['Road_traffic_density'] = df1['Road_traffic_density'].astype('category')
df1['Type_of_order'] = df1['Type_of_order'].astype('category')
df1['Type_of_vehicle'] = df1['Type_of_vehicle'].astype('category')
df1['City'] = df1['City'].astype('category')
df1['Time_taken(min)'] = df1['Time_taken(min)'].astype('int64')
df1['Festival'] = df1['Festival'].astype('category')
df1['multiple_deliveries'] = df1['multiple_deliveries'].astype('category')
df1['Vehicle_condition'] = df1['Vehicle_condition'].astype('category')

# adding new columns
df1['Time_diff'] = pd.to_datetime(df1['Time_Order_picked']) - pd.to_datetime(df1['Time_Orderd'])
df1['Time_diff'] = df1['Time_diff'].dt.seconds // 60
df['Order_datetime'] = df1.apply(
    lambda x : 
    pd.to_datetime(
    x['Order_Date'].strftime('%d-%m-%Y') + ' ' +
    x['Time_Orderd']
    , format='%d-%m-%Y %H:%M:%S'), axis = 1
)



In [57]:
df1.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weatherconditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken(min),Order_datetime,Time_diff
0,0x4607,INDORES13DEL02,37,4.9,22.745049,75.892471,22.765049,75.912471,2022-03-19,11:30:00,11:45:00,Sunny,High,2,Snack,motorcycle,0,No,Urban,24,2022-03-19 11:30:00,15
1,0xb379,BANGRES18DEL02,34,4.5,12.913041,77.683237,13.043041,77.813237,2022-03-25,19:45:00,19:50:00,Stormy,Jam,2,Snack,scooter,1,No,Metropolitian,33,2022-03-25 19:45:00,5
2,0x5d6d,BANGRES19DEL01,23,4.4,12.914264,77.678400,12.924264,77.688400,2022-03-19,08:30:00,08:45:00,Sandstorms,Low,0,Drinks,motorcycle,1,No,Urban,26,2022-03-19 08:30:00,15
3,0x7a6a,COIMBRES13DEL02,38,4.7,11.003669,76.976494,11.053669,77.026494,2022-04-05,18:00:00,18:10:00,Sunny,Medium,0,Buffet,motorcycle,1,No,Metropolitian,21,2022-05-04 18:00:00,10
4,0x70a2,CHENRES12DEL01,32,4.6,12.972793,80.249982,13.012793,80.289982,2022-03-26,13:30:00,13:45:00,Cloudy,High,1,Snack,scooter,1,No,Metropolitian,30,2022-03-26 13:30:00,15


In [21]:
df1.shape

(41419, 21)

In [22]:
df1.dtypes

ID                                     object
Delivery_person_ID                     object
Delivery_person_Age                     int64
Delivery_person_Ratings               float64
Restaurant_latitude                   float64
Restaurant_longitude                  float64
Delivery_location_latitude            float64
Delivery_location_longitude           float64
Order_Date                     datetime64[ns]
Time_Orderd                            object
Time_Order_picked                      object
Weatherconditions                    category
Road_traffic_density                 category
Vehicle_condition                    category
Type_of_order                        category
Type_of_vehicle                      category
multiple_deliveries                  category
Festival                             category
City                                 category
Time_taken(min)                         int64
Time_diff                               int64
dtype: object

In [ ]:
df.info()

In [ ]:
df.describe().T